<a href="https://colab.research.google.com/github/Param-Bhatt/Search-engine/blob/document/IR_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Library for corpus to get corpus
!pip install datasets

     |████████████████████████████████| 153kB 4.5MB/s 
     |████████████████████████████████| 245kB 16.3MB/s 
     |████████████████████████████████| 17.7MB 200kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
!pip install pdoc3

     |████████████████████████████████| 92kB 3.9MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
  Created wheel for pdoc3: filename=pdoc3-0.9.1-cp36-none-any.whl size=115994 sha256=a9756e8db9cb5a49791cbec5471832c75dbf2e9a04805ec46dd7077a98256dfa
  Stored in directory: /root/.cache/pip/wheels/84/0d/9e/b10defcfc3a3b76999ea945f0d57047340456e966e2184a451
Successfully built pdoc3


In [ ]:
!pdoc --html Embeddings_Retrieval.py

/usr/local/lib/python3.6/dist-packages/IPython/utils/module_paths.py:29: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2020-10-24 19:26:54.323476: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<unknown>:15: DeprecationWarning: invalid escape sequence \[
html/Embeddings_Retrieval.html


In [ ]:
#Library to get Bert Embeddings
!pip install bert-embedding --no-deps

In [ ]:
#Dependency of Mxnet
!pip install gluonnlp

     |████████████████████████████████| 348kB 4.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588533 sha256=ef8605f85e61e8a29b866baa29b4f62a947dcc7f964d87cf1b8395d06221b718
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
ERROR: bert-embedding 1.0.1 requires mxnet==1.4.0, which is not installed.
ERROR: bert-embedding 1.0.1 requires typing==3.6.6, which is not installed.
ERROR: bert-embedding 1.0.1 has requirement gluonnlp==0.6.0, but you'll have gluonnlp 0.10.0 which is incompatible.
ERROR: bert-embedding 1.0.1 has requirement numpy==1.14.6, but you'll have numpy 1.18.5 which is incompatible.


In [ ]:
#Interface the cuda API of the gpu decrease the run time from almost 5 hours to 7 minutes
!pip install mxnet-cu101

In [ ]:
import re                                   #Regular Expression
import nltk                                 #For Lemmatizationa and stopwords removal
import mxnet as mx                          #For interfacing cuda 
import numpy as np                          #Array Data Structure and Cosine Score
from datasets import load_dataset           #load_dataset function form 'dataset' library to load dataset
from nltk.corpus import stopwords           #Get stopwords
from bert_embedding import BertEmbedding    #BertEmbeddings function gives embeddings
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
STOPWORDS = set(stopwords.words('english'))
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;!@]')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess(text):
    """
    string: a string\n
    return: preprocessed string\n
    Takes text as input and \n
        1) Makes everything lower case.\n
        2) Deletes '[/(){}\[\]\|@,;!@]' and then '[^0-9a-z #+_]'.\n
        3) Removes StopWords.\n
        4) Lemmatizes every word.\n
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(" " , text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub("", text)         # delete symbols which are in BAD_SYMBOLS_RE from text
    textlist = text.split()
    text = [wrd for wrd in textlist if wrd not in STOPWORDS]
    text = " ".join(text)
    
    #Lemmatize
    lem_words = []
    for word in text.split():
        lem_words.append(lemmatizer.lemmatize(word))
    return " ".join(lem_words)

In [ ]:
def load_dataset_and_preprocess():
    """
    Input:None\n
    Load dataset and preprocess it\n
    Output:Dataset and preprocessed Dataset\n
    """
    dataset = load_dataset("aeslc")
    """Load the Dataset"""
    emails = dataset['train'][:]['email_body']
    """Training Emails in the dataset"""
    #unpreprocessed
    email_sent = []
    """Array of sentences in the emails"""
    for i in range(len(emails)):
        email_sent.append([i for i in emails[i].split('\n') if len(i)!=0 ])

    #preprocessed
    corpus = []
    """Hold the Preprocessed Corpus"""
    for email in emails:
        email = email.split("\n")
        preprocessed_sentence=[]
        for i in range(len(email)):
            if len(email[i]) > 0:
                prep_s = preprocess(email[i])
                if len(prep_s) > 0:
                    preprocessed_sentence.append(prep_s)
        corpus.append(preprocessed_sentence)
    return email_sent, corpus

# BERT

In [ ]:
def build_bert_embeddings_index(corpus):
    """
    Input: Dataset as corpus\n
    Build BERT Embeddings of sentences averaged over words in each sentence and index them\n
    Output: Embeddings of each sentence in Dataset\n
    """
    try:
        ctx = mx.gpu(0)
        """Set context to GPU on Mxnet"""
        be = BertEmbedding(ctx=ctx)
        """Function to get embeddings of each word in array of sentences """
    except:
        be = BertEmbedding()
    embeddings = []
    """Store all the Sentence Embeddings in the corpus"""
    for email in range(len(corpus)):
        printc=True
        for i in be(corpus[email]):
            if email%500==0 and printc:
                print("Iteration: ", email)
                printc=False
            try:
                embeddings.append(sum(i[1])/len(i[1]))
            except:
                print(i, email, corpus[email])
    return embeddings, be

In [ ]:
def build_reference(corpus):
    """
    Input: Dataset as corpus\n
    Build reference of each line to email and starting line of the email.\n
    Output: Reference index from line to email \n
    """
    indx_to_email = dict()
    """Dictionary to refer the corpus line number (l) to the Email number and the fist line number of the email"""
    l=0
    for indx_email in range(len(corpus)):
        start = l
        for j in range(len(corpus[indx_email])):
            indx_to_email[l]=(start, indx_email)
            l+=1
    return indx_to_email

In [ ]:
def find_email(i_email, unprep, highlight_line=None):
    """
    Input:Index of email and the email dataset, optional line number to highlight\n
    Prints the email with optional highlighted line\n
    Output: Prints the same\n
    """
    for i in range(len(unprep[i_email])):
        if i==highlight_line:
            print('\33[34m' + unprep[i_email][i] + '\033[0m')
        else:
            print(unprep[i_email][i])

In [ ]:
def cosine_score(vec1, vec2):
    """
    Input: Two embeddings of same dimentions\n
    Finds the cosine score among them\n
    Output: Cosine Similarity\n
    """
    try:
        if vec1.all() == None or vec2.all() == None:
            return 0
    except: 
        return 0
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def bert_search(query, embeddings, corpus, indx_to_email, top_n=3, is_preprocess=False):  
    """
    Input:Index of email and the email dataset, optional line number to highlight\n
    Prints the email with optional highlighted line\n
    Output: Prints the same\n
    """  
    try:
        ctx = mx.gpu(0)
        be = BertEmbedding(ctx=ctx)
    except:
        be = BertEmbedding()

    l = 0
    q = query

    c = [i for i in q.split('\n') if len(i)!=0]
    """Split query per line and then """

    if is_preprocess:
        que = [preprocess(i) for i in q.split('\n') if len(i)!=0 ] 
    else: 
        que = q.split('\n')
    
    embedding = be(que)
    """Array to store the splitted words in the query and their corresponding embeddings"""
    for i in range(len(embedding)):
        print("\33[35mQuery {} of {} \033[0m".format(l+1, len(embedding)))
        e = embedding[i][1]
        """Array of embeddings of each word in the sentence"""
        sent_embedding = sum(e)/len(e)
        """Averaged embedding of the sentence"""
        similarity = []
        """Array to store the similarity with embeddings of each sentence in the corpus"""
        for j in embeddings:
            similarity.append(cosine_score(sent_embedding, j))

        sim = np.argsort(similarity)[::-1][:top_n]
        """Array to store the top n similarities"""
        print("Query:", "\33[31m" + c[l] + "\033[0m")
        if is_preprocess:
            print("Preprocessed query:", "\33[31m" + que[i]+ "\033[0m" "\n")
        l+=1
        for i in sim:
            print("\33[33m" + "Cosine Similarity: {:.3f} \t Email {} \t Line {} \033[0m".format(similarity[i], indx_to_email[i][1], i-indx_to_email[i][0]))
            print("\33[34m" + str(corpus[indx_to_email[i][1]][i-indx_to_email[i][0]]) + '\033[0m',  "\n")
        

In [ ]:
%%time
unprep, prep = load_dataset_and_preprocess()    #Load Dataset

Using custom data configuration default


Dataset aeslc downloaded and prepared to /root/.cache/huggingface/datasets/aeslc/default/1.0.0/0f335d7ea7f9ab9e325673a86c6aa0e2cd503c5dbf17d48fddcb02840a92fc96. Subsequent calls will reuse this data.
CPU times: user 7.74 s, sys: 1.11 s, total: 8.85 s
Wall time: 11.2 s


In [ ]:
%%time
#Build Index for unproprocessed Data
unprep_index, be 		= build_bert_embeddings_index(unprep)

#Building reference
unprep_indx_to_email    = build_reference(unprep)

Vocab file is not found. Downloading.
Iteration:  0
Iteration:  500
Iteration:  1000
Iteration:  1500
Iteration:  2000
Iteration:  2500
Iteration:  3000
Iteration:  3500
Iteration:  4000
Iteration:  4500
Iteration:  5000
Iteration:  5500
Iteration:  6000
Iteration:  6500
Iteration:  7000
Iteration:  7500
Iteration:  8000
Iteration:  8500
Iteration:  9000
Iteration:  9500
Iteration:  10000
Iteration:  10500
Iteration:  11000
Iteration:  11500
Iteration:  12000
Iteration:  12500
Iteration:  13000
Iteration:  13500
Iteration:  14000
CPU times: user 10min 29s, sys: 1min 27s, total: 11min 56s
Wall time: 7min 35s


In [ ]:
%%time
#Build Index for preprocessed Data
preprocessed_index, be 	        = build_bert_embeddings_index(prep)

#Building reference
prep_indx_to_email 	            = build_reference(prep)

Iteration:  0
Iteration:  500
Iteration:  1000
Iteration:  1500
Iteration:  2000
Iteration:  2500
Iteration:  3000
Iteration:  3500
Iteration:  4000
Iteration:  4500
Iteration:  5000
Iteration:  5500
Iteration:  6000
Iteration:  6500
Iteration:  7000
Iteration:  7500
Iteration:  8000
Iteration:  8500
Iteration:  9000
Iteration:  9500
Iteration:  10000
Iteration:  10500
Iteration:  11000
Iteration:  11500
Iteration:  12000
Iteration:  12500
Iteration:  13000
Iteration:  13500
Iteration:  14000
CPU times: user 10min 16s, sys: 1min 25s, total: 11min 42s
Wall time: 7min 1s


In [ ]:
%%time
#Save Data to PKL file
import pickle

with open('preprocessed_index.pkl', 'wb') as fp:
    pickle.dump(preprocessed_index, fp) 

with open('unprep_index.pkl', 'wb') as fp:
    pickle.dump(unprep_index, fp) 
len(unprep_index), len(preprocessed_index)

CPU times: user 1.74 s, sys: 1.26 s, total: 2.99 s
Wall time: 3.83 s


In [ ]:
%%time
import pickle
with open ('preprocessed_index.pkl', 'rb') as fp:
    preprocessed_index = pickle.load(fp)

with open ('unprep_index.pkl', 'rb') as fp:
    unprep_index = pickle.load(fp)
len(unprep_index), len(preprocessed_index)

CPU times: user 797 ms, sys: 316 ms, total: 1.11 s
Wall time: 1.12 s


#Glove

In [ ]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

--2020-10-24 19:11:43--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1       49%[========>           ] 406.11M  1.93MB/s    eta 3m 5s  ^C


In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def init_glove_embeddings(dimentions=300):
    """
    Input: Dimentions of the Embeddings to use\n
    Builds the glove embeddings in a dicyionary from the file\n
    Output: Dictionary of the pretrained dictionary\n
    """
    glove_embeddings = {}
    """Dictionary to store the embeddings from Glove file"""
    with open("glove.6B."+str(dimentions)+"d.txt", 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            glove_embeddings[word] = vector
    return glove_embeddings

In [ ]:
def glove_embed(sent):
    """
    Input: Sentence as string\n
    Finds the glove embeddings of each word and adds it for each sentence\n
    Output: Embeddings and the length of words taken to form the embeddings\n
    """
    sent = sent.lower()
    embeddings = [0 for _ in embeddings_dict["king"]]
    """Array to store query embeddings initialized to 0"""
    length = 0
    for word in sent.split():
        try:
            embeddings+=embeddings_dict[word]
            length+=1
        except:
            pass
            
    return embeddings, length

In [ ]:
def build_glove_index(corpus):
    """
    Input: Corpus\n
    Builds the index of embeddings of each sentence in the corpus\n
    Output: Embddeings Index of each sentence in the corpus\n
    """
    embeddings = []
    """Array to store the sentence Embeddings of the dataset"""
    sentence_corpus = []
    """Array of strings to store the sentence in the corpus"""
    for email in range(len(corpus)):
        printc=True
        for i in corpus[email]:
            sentence_corpus.append(i)
            j, length = glove_embed(i)
            if length<= 0:
                embeddings.append(None)
                continue
            if email%1000==0 and printc:
                print("Iteration: ", email)
                printc=False
            try:
                embeddings.append(j/length)
            except:
                print(i, email)
    assert len(embeddings) == len(sentence_corpus)
    return embeddings, sentence_corpus

In [ ]:
def glove_search(query, embeddings, sentence_corpus, indx_to_email, is_preprocess=False, top_n=2):
    """
    Input: query to search, Sentence corpus to search in , top n results\n
    Searches with cosing similarity among the embeddings\n
    Output: most similar items to query\n
    """
    q_emb = []
    """Store the Embddings of the query"""
    print("Query:", "\33[31m" + query + "\033[0m")
    if is_preprocess:
        query=preprocess(query)
        print("Preprocessed Query:", "\33[31m" + query + "\033[0m ")
    j, length = glove_embed(query.lower())
    
    if length<= 0:
        print("Sent not found",  i)
        q_emb.append(None)
    try:
        q_emb.append(j/length)
    except:
        print(i, email)
    similarity = []
    for emb in range(len(embeddings)):
        score = cosine_score(embeddings[emb], q_emb[0])
        similarity.append(cosine_score(embeddings[emb], q_emb[0]))
    sim = np.argsort(similarity)[::-1][:top_n]
    for i in sim:
            print("\33[33m" + "Cosine Similarity: {:.3f} \t Email {} \t Line {} \033[0m".format(similarity[i], indx_to_email[i][1], i-indx_to_email[i][0]))
            print("\33[34m" + str(sentence_corpus[indx_to_email[i][1]][i-indx_to_email[i][0]]) + '\033[0m',  "\n")

In [ ]:
%%time
embeddings_dict  = init_glove_embeddings()
unprep, prep = load_dataset_and_preprocess()
prep_embeddings, prep_sentence = build_glove_index(prep)

Using custom data configuration default
Reusing dataset aeslc (/root/.cache/huggingface/datasets/aeslc/default/1.0.0/0f335d7ea7f9ab9e325673a86c6aa0e2cd503c5dbf17d48fddcb02840a92fc96)


Iteration:  0
Iteration:  1000
Iteration:  2000
Iteration:  3000
Iteration:  4000
Iteration:  5000
Iteration:  6000
Iteration:  7000
Iteration:  8000
Iteration:  9000
Iteration:  10000
Iteration:  11000
Iteration:  12000
Iteration:  13000
Iteration:  14000
CPU times: user 35.3 s, sys: 809 ms, total: 36.1 s
Wall time: 36.5 s


In [ ]:
%%time
unprep, prep = load_dataset_and_preprocess()

Using custom data configuration default
Reusing dataset aeslc (/root/.cache/huggingface/datasets/aeslc/default/1.0.0/0f335d7ea7f9ab9e325673a86c6aa0e2cd503c5dbf17d48fddcb02840a92fc96)


CPU times: user 4.07 s, sys: 14.8 ms, total: 4.08 s
Wall time: 4.43 s


In [ ]:
%%time
prep_embeddings, prep_sentence = build_glove_index(prep)

Iteration:  0
Iteration:  1000
Iteration:  2000
Iteration:  3000
Iteration:  4000
Iteration:  5000
Iteration:  6000
Iteration:  7000
Iteration:  8000
Iteration:  9000
Iteration:  10000
Iteration:  11000
Iteration:  12000
Iteration:  13000
Iteration:  14000
CPU times: user 7.26 s, sys: 76.9 ms, total: 7.34 s
Wall time: 7.34 s


In [ ]:
%%time
embeddings_dict  = init_glove_embeddings()
unprep, prep = load_dataset_and_preprocess()
unprep_embeddings, unprep_sentence = build_glove_index(unprep)

Using custom data configuration default
Reusing dataset aeslc (/root/.cache/huggingface/datasets/aeslc/default/1.0.0/0f335d7ea7f9ab9e325673a86c6aa0e2cd503c5dbf17d48fddcb02840a92fc96)


Iteration:  0
Iteration:  1000
Iteration:  2000
Iteration:  3000
Iteration:  4000
Iteration:  5000
Iteration:  6000
Iteration:  7000
Iteration:  8000
Iteration:  9000
Iteration:  10000
Iteration:  11000
Iteration:  12000
Iteration:  13000
Iteration:  14000
CPU times: user 36.5 s, sys: 492 ms, total: 36.9 s
Wall time: 37.2 s


In [ ]:
#Building reference
unprep_indx_to_email    = build_reference(unprep)

prep_indx_to_email 	            = build_reference(prep)

# query = unprep[10007][3]
# query = unprep[10069][3]



In [ ]:
%%time
query = "I want the work to be submitted by this weekend"
glove_search(query, unprep_embeddings, unprep, unprep_indx_to_email, is_preprocess=False, top_n=4)

Query: I want the work to be submitted by this weekend
Cosine Similarity: 0.962 	 Email 8029 	 Line 1 
They want the regular ones to be done by the end of next week, and  Sherry's has to be done by Jan 19, 2001. 

Cosine Similarity: 0.962 	 Email 13139 	 Line 3 
I will be out of the office on Friday and Monday, and so if you wish to discuss the presentation with me beforehand, it will be best if we speak by Thursday. 

Cosine Similarity: 0.961 	 Email 4764 	 Line 3 
If this is acceptable, I will finalize the agreement, get it signed and have it forwarded to you for counter-signature by DB. 

Cosine Similarity: 0.961 	 Email 13135 	 Line 10 
We are hoping to get this approved by Wednesday Tokyo time so if this needs to be passed on, I would be grateful if you could do this today (your time). 

CPU times: user 3.34 s, sys: 7.58 ms, total: 3.34 s
Wall time: 3.33 s


In [ ]:
%%time
glove_search(query, prep_embeddings, prep, prep_indx_to_email, is_preprocess=True, top_n=4)

Query: I want the work to be submitted by this weekend
Preprocessed Query: want work submitted weekend 
Cosine Similarity: 0.878 	 Email 2048 	 Line 1 
want work evening 

Cosine Similarity: 0.872 	 Email 8202 	 Line 3 
please let know time work tomorrow friday would work best schedule 

Cosine Similarity: 0.868 	 Email 12551 	 Line 2 
would like try get issue addressed next week people need talk turn draft isda schedule please make priority 

Cosine Similarity: 0.868 	 Email 8974 	 Line 2 
work weekend 

CPU times: user 3.16 s, sys: 6.6 ms, total: 3.17 s
Wall time: 3.16 s


In [ ]:
%%time
bert_search(query, unprep_index, unprep, unprep_indx_to_email, top_n=4, is_preprocess=False)

Query 1 of 1 
Query: I want the work to be submitted by this weekend
Cosine Similarity: 0.843 	 Email 9468 	 Line 2 
I just want  to make sure it will be available before March bidweek. 

Cosine Similarity: 0.840 	 Email 506 	 Line 6 
I will schedule an August date with you  for your review as soon as I know the exact date of the final PRC meeting. 

Cosine Similarity: 0.835 	 Email 951 	 Line 4 
This should be completed early next week and either your group or Steve will get it circulated for sign-offs. 

Cosine Similarity: 0.831 	 Email 6554 	 Line 1 
We are putting together a symposium brochure this month and would like to receive 2 items from you by Wednesday, December 12, 2001. 

CPU times: user 2.64 s, sys: 417 ms, total: 3.06 s
Wall time: 2.89 s


In [ ]:
%%time
bert_search(query, preprocessed_index, prep, prep_indx_to_email, top_n=4, is_preprocess=True)

In [ ]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
print("Variable with their size in Bytes")
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))